# Aaron's ISR challenge - are amp boundaries visible after ISR?
Craig Lage 09-Jan-23

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

from lsst.summit.utils import quickSmooth
%matplotlib inline

from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfTask

In [ ]:
def plotExp(exp):
    # Borrowed from summit utils
    data = quickSmooth(exp.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure

def repairCosmics(postIsr):
    # Borrowed from summit utils
    if postIsr.getPsf() is None:
        installPsfTask = InstallGaussianPsfTask()
        installPsfTask.run(postIsr)

    # TODO: try adding a reasonably wide Gaussian as a temp PSF
    # and then just running repairTask on its own without any
    # imChar. It should work, and be faster.
    repairConfig = CharacterizeImageTask.ConfigClass()
    repairConfig.doMeasurePsf = False
    repairConfig.doApCorr = False
    repairConfig.doDeblend = False
    repairConfig.doWrite = False
    repairConfig.repair.cosmicray.nCrPixelMax = 200000
    repairTask = CharacterizeImageTask(config=repairConfig)
    repairTask.repair.run(postIsr)
    return


In [ ]:
butler = Butler("/repo/main", collections=["LSSTCam/raw/all","u/cslage/calib/13144/calib.20220107"])
biasButler = Butler("/repo/main", collections=["u/cslage/bias_parallel_13144_B_17jan23"])

In [ ]:
expId = 3021120700218
detector = 55

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doLinearize=True
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=True
isrConfig.doBrighterFatter=False
isrConfig.doDark=True
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=True
isrConfig.usePtcGains=True
isrConfig.doDefect=True
isrConfig.doDeferredCharge=True
isrConfig.doNanMasking=True
isrConfig.doInterpolate=True
isrConfig.doSaturation=True
isrConfig.doSaturationInterpolation=True
isrConfig.doWrite=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
exp = butler.get('raw', detector=detector, exposure=expId, instrument="LSSTCam")
bias = biasButler.get('bias', detector=detector, exposure=expId, instrument="LSSTCam")
flat = butler.get('flat', detector=detector, exposure=expId, instrument="LSSTCam")
dark = butler.get('dark', detector=detector, exposure=expId, instrument="LSSTCam")
defects = butler.get('defects', detector=detector, exposure=expId, instrument="LSSTCam")
ptc = butler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")
#ptc.gain['C10'] /= corr_c10
linearizer = butler.get('linearizer', detector=detector, exposure=expId, instrument="LSSTCam")
deferredChargeCalib = butler.get('cpCtiCalib', detector=detector, exposure=expId, instrument="LSSTCam")


isrResult = isrTask.run(exp, bias=bias, flat=flat, dark=dark, defects=defects, \
                        ptc=ptc, linearizer=linearizer, deferredChargeCalib=deferredChargeCalib)

postIsrExp = isrResult.exposure
repairCosmics(postIsrExp)
fig = plotExp(postIsrExp)
plt.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/Challenge_{detector}_{expId}_17Jan23.png")#

In [ ]:
fig = plt.figure()
plt.subplots_adjust(hspace=0.5, wspace=0.5)

rawPlot = np.median(isrResult.exposure.image.array[2100:3900, 0:4000], axis=0)
#rawPlot -= np.median(rawPlot)
xplot1 = np.arange(520, 3500)
vals = np.polyfit(xplot, rawPlot[520:3500], 1)
xplot2 = np.arange(0, 4000)
yplot = xplot2 * vals[0] + vals[1]
corr_c10 = np.median((rawPlot/yplot)[20:480])
print(corr_c10)
plt.subplot(2,1,1)
plt.title(f"Median Flat Level - Top")
plt.plot(rawPlot, label="PostISR")
plt.plot(xplot2, yplot, color='red', ls='--')
for i in range(8):
    x = 509 * i
    ymin = -10
    ymax = 10
    plt.plot([x, x], [ymin, ymax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(1400, 1520)
#plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
rawPlot = np.median(isrResult.exposure.image.array[100:1900, 0:4000], axis=0)
rawPlot -= np.median(rawPlot)
plt.subplot(2,1,2)
plt.title(f"Median Flat Level - Bottom")
plt.plot(rawPlot, label="PostISR")
for i in range(8):
    x = 509 * i
    ymin = -10
    ymax = 10
    plt.plot([x, x], [ymin, ymax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(-50,50)
#plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")

fig.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/Challenge_{detector}_{expId}_Scan_17Jan23.png")

In [ ]:
np.median((rawPlot/yplot)[20:480])

In [ ]:
for i in range(20,500):
    print(rawPlot[i], yplot[i], rawPlot[i]/yplot[i])

In [ ]:
fig = plt.figure()
plt.subplots_adjust(hspace=0.5, wspace=0.5)

rawPlot = np.median(isrResult.exposure.image.array[2100:3900, 0:4000], axis=0)
rawPlot -= np.median(rawPlot)
biasPlot = np.median(bias.image.array[2100:3900, 0:4000], axis=0)
biasPlot -= np.median(biasPlot)
plt.subplot(2,1,1)
plt.title(f"Parallel Overscan Debug - Top")
plt.plot(rawPlot, label="Raw")
plt.plot(biasPlot, ls = '--', label="MasterBias")
for i in range(8):
    x = 509 * i
    ymin = -10
    ymax = 10
    plt.plot([x, x], [ymin, ymax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(-50,50)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
rawPlot = np.median(isrResult.exposure.image.array[100:1900, 0:4000], axis=0)
rawPlot -= np.median(rawPlot)
biasPlot = np.median(bias.image.array[100:1900, 0:4000], axis=0)
biasPlot -= np.median(biasPlot)
plt.subplot(2,1,2)
plt.title(f"Parallel Overscan Debug - Bottom")
plt.plot(rawPlot, label="Raw")
plt.plot(biasPlot, ls = '--', label="MasterBias")
for i in range(8):
    x = 509 * i
    ymin = -10
    ymax = 10
    plt.plot([x, x], [ymin, ymax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(-50,50)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")

#fig.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/Challenge_{detector}_{expId}_Scan_Bias_09Jan23.png")

In [ ]:
fig = plt.figure()
plt.subplots_adjust(hspace=0.5, wspace=0.5)

rawPlot = np.median(isrResult.exposure.image.array[2100:3900, 0:4000], axis=0)
rawPlot -= np.median(rawPlot)
biasPlot = np.median(bias.image.array[2100:3900, 0:4000], axis=0)
biasPlot -= np.median(biasPlot)
plt.subplot(2,1,1)
plt.title(f"Parallel Overscan Debug - Top")
plt.plot(rawPlot, label="Raw")
plt.plot(biasPlot, ls = '--', label="MasterBias")
for i in range(8):
    x = 509 * i
    ymin = -10
    ymax = 10
    plt.plot([x, x], [ymin, ymax], ls = '--', lw=1, color = 'black')
plt.xlim(450,550)
plt.ylim(-50,50)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
rawPlot = np.median(isrResult.exposure.image.array[100:1900, 0:4000], axis=0)
rawPlot -= np.median(rawPlot)
biasPlot = np.median(bias.image.array[100:1900, 0:4000], axis=0)
biasPlot -= np.median(biasPlot)
plt.subplot(2,1,2)
plt.title(f"Parallel Overscan Debug - Bottom")
plt.plot(rawPlot, label="Raw")
plt.plot(biasPlot, ls = '--', label="MasterBias")
for i in range(8):
    x = 509 * i
    ymin = -10
    ymax = 10
    plt.plot([x, x], [ymin, ymax], ls = '--', lw=1, color = 'black')
plt.xlim(450,550)
plt.ylim(-50,50)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")

#fig.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/Challenge_{detector}_{expId}_Scan_Bias_Blowup_09Jan23.png")

## Let's look at just one of the bias images

In [ ]:
expId = 3021120600560
detector = 74

In [ ]:
exp = butler.get('raw', detector=detector, exposure=expId, instrument="LSSTCam")

In [ ]:
rawPlot = np.median(exp.image.array[2100:3900, 0:540], axis=0)
#rawPlot -= np.median(rawPlot)
ovPlot = np.median(exp.image.array[4000:4047, 0:540], axis=0)
#biasPlot -= np.median(biasPlot)
plt.plot(rawPlot)
plt.plot(ovPlot)

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=True
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.usePtcGains=False
isrConfig.doDefect=False
isrConfig.doDeferredCharge=False
isrConfig.doNanMasking=False
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrConfig.doWrite=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
isrResult = isrTask.run(exp)
postIsrExp = isrResult.exposure

In [ ]:
isrPlot = np.median(postIsrExp.image.array[2100:3900, 0:4000], axis=0)
plt.plot(isrPlot)


## Let's look at just one of the flat images

In [ ]:
expId = 3021120700218
detector = 74

In [ ]:
exp = butler.get('raw', detector=detector, exposure=expId, instrument="LSSTCam")

In [ ]:
rawPlot = np.median(exp.image.array[2100:3900, 0:540], axis=0)
#rawPlot -= np.median(rawPlot)
ovPlot = np.median(exp.image.array[4000:4047, 0:540], axis=0)
#biasPlot -= np.median(biasPlot)
plt.plot(rawPlot)
plt.plot(ovPlot)

In [ ]:
# I think the dip at the beginning of the row is caused by CTI
rawPlot = np.median(exp.image.array[2100:3900, 0:540], axis=0)
#rawPlot -= np.median(rawPlot)
ovPlot = np.median(exp.image.array[4000:4047, 0:540], axis=0)
#biasPlot -= np.median(biasPlot)
plt.plot(rawPlot)
plt.plot(ovPlot)
plt.xlim(64, 100)

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=True
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.usePtcGains=False
isrConfig.doDefect=False
isrConfig.doDeferredCharge=False
isrConfig.doNanMasking=False
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrConfig.doWrite=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
isrResult = isrTask.run(exp)
postIsrExp = isrResult.exposure

In [ ]:
isrPlot = np.median(postIsrExp.image.array[2100:3900, 0:509], axis=0)
plt.plot(isrPlot)


In [ ]:
corrs = correctGains(postIsrExp)

In [ ]:
corrs

In [ ]:
#rawPlot -= np.median(rawPlot)

vals = np.polyfit(xplot, rawPlot[520:3500], 1)
xplot2 = np.arange(0, 4000)
yplot = xplot2 * vals[0] + vals[1]
corr_c10 = np.median((rawPlot/yplot)[20:480])
print(corr_c10)
plt.subplot(2,1,1)
plt.title(f"Median Flat Level - Top")
plt.plot(rawPlot, label="PostISR")
plt.plot(xplot2, yplot, color='red', ls='--')
for i in range(8):
    x = 509 * i
    ymin = -10
    ymax = 10
    plt.plot([x, x], [ymin, ymax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(1400, 1520)
#plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
rawPlot = np.median(isrResult.exposure.image.array[100:1900, 0:4000], axis=0)
rawPlot -= np.median(rawPlot)
plt.subplot(2,1,2)
plt.title(f"Median Flat Level - Bottom")
plt.plot(rawPlot, label="PostISR")
for i in range(8):
    x = 509 * i
    ymin = -10
    ymax = 10
    plt.plot([x, x], [ymin, ymax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(-50,50)
#plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")

fig.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/Challenge_{detector}_{expId}_Scan_17Jan23.png")

In [ ]:
for amp in exp.getDetector().getAmplifiers():
    print(amp.getName(), amp.getBBox())
    bbox = amp.getBBox()

In [ ]:
bbox.beginX

In [ ]:
np.arange((0,10))